# Writing data into PostgreSQL

### Installing psycopg2

In [ ]:
%pip install psycopg2-binary

In [ ]:
import psycopg2
import psycopg2.extras
import json

### cars_silver table DDL

```SQL
CREATE TABLE public.cars_silver (
	id integer NOT NULL,
	brand varchar NOT NULL,
	power integer,
	num_cylinders integer,
	block_type varchar,
	top_speed_value integer,
	top_speed_unit varchar,
	CONSTRAINT cars_bronze_pk PRIMARY KEY (id)
);
```

In [ ]:
conn_string = f"host=localhost dbname=mentorship " \
              f"user=postgres password=12345678"

query = """
INSERT INTO cars_silver(id, brand, power, num_cylinders, block_type, top_speed_value, top_speed_unit)
VALUES (%(id)s, %(brand)s, %(power)s, %(num_cylinders)s, %(block_type)s, %(top_speed_value)s, %(top_speed_unit)s)
ON CONFLICT (id)
DO NOTHING
"""

In [ ]:
def convert_power_to_int(row, new_row):
    try:
        new_row["power"] = int(row["power"])
    except:
        new_row["power"] = None


def break_cylinders_column(row, new_row):
    old_cylinders = row.get("cylinders")
    if old_cylinders:
        if ', ' in old_cylinders:
            new_row["num_cylinders"] = int(old_cylinders.split(', ')[0])
            new_row["block_type"] = old_cylinders.split(', ')[1]
        elif old_cylinders.isnumeric():
            new_row["num_cylinders"] = int(old_cylinders)
            new_row["block_type"] = None
        else:
            new_row["num_cylinders"] = None
            new_row["block_type"] = old_cylinders
    else:        
        new_row["num_cylinders"] = None
        new_row["block_type"] = None    


def break_top_speed_column(row, new_row):
    old_top_speed = row.get("top_speed")
    try:
        if old_top_speed:
            if ' ' in old_top_speed:
                    new_row["top_speed_value"] = int(old_top_speed.split(' ')[0])
                    new_row["top_speed_unit"] = old_top_speed.split(' ')[1]
            elif old_top_speed.isnumeric():
                new_row["top_speed_value"] = int(old_top_speed)
                new_row["top_speed_unit"] = None
        else:
            new_row["top_speed_value"] = None
            new_row["top_speed_unit"] = None    
    except:
        new_row["top_speed_value"] = None
        new_row["top_speed_unit"] = None

In [ ]:
with open('input/cars.json') as json_file:
    data = json.load(json_file)

In [ ]:
data_to_insert = list()
for row in data:
    new_row = dict()
    new_row["id"] = row["car_id"]
    new_row["brand"] = row["brand"]
    convert_power_to_int(row, new_row)
    break_cylinders_column(row, new_row)
    break_top_speed_column(row, new_row)
    data_to_insert.append(new_row)

In [ ]:
# We use `with` to garantee that the conn will be closed as soon as the commands within the block are run

with psycopg2.connect(conn_string) as conn:
    cursor = conn.cursor()
    psycopg2.extras.execute_batch(cursor, query, data_to_insert)
    conn.commit()